This is a classification example to show how to use Oboe for training and testing, in the context of AutoML.

In [1]:
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#import Oboe modules; will be simplified after Oboe is made pip installable
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [3]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [4]:
#load dataset for meta-training and meta-testing
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
#autolearner arguments
autolearner_kwargs = {
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
}

In [6]:
#intialize the autolearner class
m = AutoLearner('classification', **autolearner_kwargs)

In [7]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:128: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]
../automl/convex_opt.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:128: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]


In [8]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.03968253968253967
elapsed time: 7.2537841796875


In [9]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'KNN': [{'n_neighbors': 15, 'p': 1},
   {'n_neighbors': 1, 'p': 2},
   {'n_neighbors': 1, 'p': 2}]}}